In [29]:
import re
import pandas as pd
import nltk
from collections import defaultdict
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs

In [30]:
# using data frames to read bag of joy words
df = pd.read_csv("joy_words.txt", encoding="utf-8", index_col=None , names=["joy_words"])
joy_words = df["joy_words"].str.lower().to_list()

wr_directory = 'joy_tweets_testset_test.txt'

In [61]:
# reading tweets as a df
# error_bad_lines to skip bad formatted lines
# namses to assign column names after separation
original_sentences_df = pd.read_csv(
"tweets_trainset_short.txt", 
    sep="\t", 
    error_bad_lines=False, 
    encoding="utf-8",
    index_col=None,
    names=["id", "timestamp", "sentence", "time"])
original_sentences_df.head(10)

,id,timestamp,sentence,time
0,60730027,6320951896,@thediscovietnam coo. thanks. just dropped yo...,2009-12-03 18:41:07
1,60730027,6320673258,@thediscovietnam shit it ain't lettin me DM yo...,2009-12-03 18:31:01
2,60730027,6319871652,"@thediscovietnam hey cody, quick question...ca...",2009-12-03 18:01:51
3,60730027,6318151501,@smokinvinyl dang. you need anything? I got ...,2009-12-03 17:00:16
4,60730027,6317932721,"maybe i'm late in the game on this one, but th...",2009-12-03 16:52:36
5,60730027,6317701252,i really hope A.I. makes the most of this seco...,2009-12-03 16:44:37
6,60730027,6315879930,@smokinvinyl danny boy! wanna check out d-nice...,2009-12-03 15:39:10
7,60730027,6314008277,"...and if you have ppl that you care about, ma...",2009-12-03 14:26:33
8,60730027,6313959103,...that shit weighs heavy on me. take respons...,2009-12-03 14:24:40
9,60730027,6313919319,...including his last failed relationship. an...,2009-12-03 14:23:07


In [18]:
# regex patterns
indx = r'\@.+?\s'
html = r'(www|http:|https:)[^\s]+[\w]'
punct = r'[!"\\#\\$%\\&\\(\\)\\*\\+,\\-\\.\/:;<=>\\?@\\[\\\\\\\]\\^_`\\{\\|\\}\\~]'

In [59]:
# joining regex patterns
replace_patterns = "|".join([indx, html, punct])
minor_pattern = "\s+"

# replacing patterns, cleaning whitespaces, saving column as a string with astype(str)
original_sentences_df["sentence"] = original_sentences_df["sentence"].str.replace(replace_patterns, "", regex=True)
original_sentences_df["sentence"] = original_sentences_df["sentence"].str.replace(minor_pattern, " ", regex=True)
senteces_ready = original_sentences_df["sentence"].astype(str).str.lower().str.strip().to_list()

In [58]:
d = [] # combined list of joy and no_joy sentences
joy = [] 
no_joy = []

joy_counter = 0
no_joy_counter = 0

for sents in senteces_ready:
    sentences = sents.split(" ")
    x = set(sentences).intersection(joy_words)
    if x:
        d.append((" ".join(sentences), 'joy_word'))
        joy.append((" ".join(sentences), 'joy_word'))
        joy_counter += 1
    else:
        d.append((" ".join(sentences), 'no_joy_word'))
        no_joy.append((" ".join(sentences), 'no_joy_word'))
        no_joy_counter += 1

In [21]:
#with open(wr_directory, 'w', encoding="utf-8") as out:
    #p = "".join(str(d))
    #print(p)
    #out.write("".join(str(d)))

In [34]:
# print first 10 sentences to test
for i, line in enumerate(d):
    if i <= 10:
        print(i, line)
        pass

0 ('coo thanks just dropped you a line', 'no_joy_word')
1 ("shit it ain't lettin me dm you back what's your email", 'no_joy_word')
2 ('hey cody quick questioncan you dm me', 'no_joy_word')
3 ('dang you need anything i got some left over meds', 'no_joy_word')
4 ("maybe i'm late in the game on this one but this lowender vst is making my apt rumble", 'no_joy_word')
5 ("i really hope ai makes the most of this second chance in philly i'm glad he's goin home", 'joy_word')
6 ('danny boy wanna check out d-nice at the afex 1 year tonight we could pre-game at mine and walk over', 'no_joy_word')
7 ('and if you have ppl that you care about make sure to let them know life is too short to lose friends over bullshit peace and love', 'joy_word')
8 ("that shit weighs heavy on me take responsibility for your life i don't blame anyone for where i am in this world", 'no_joy_word')
9 ('including his last failed relationship and while i know that none of it is grounded in reality and i actually am worried a

In [57]:
# new list for splitted tockens
sents = [] 

# filtering sentences by word size and sentiments
for (words, sentiment) in joy + no_joy:
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3]
    #print(words_filtered)
    sents.append((words_filtered, sentiment))

In [37]:
# functions creating a dict with words ordered by their appearance frequency
def get_words_in_sents(sents):
    all_words = []
    for (words, sentiment) in sents:
      all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [46]:
word_features = get_word_features(get_words_in_sents(sents))
#word_features

In [56]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [40]:
# applying features to the classifier using the apply_features method and the tweets processed above
training_set = nltk.classify.apply_features(extract_features, sents)

In [45]:
# classifier training
classifier = nltk.NaiveBayesClassifier.train(training_set)

KeyboardInterrupt: 

In [42]:
# counting frequency of each label in the training set 
def train(labeled_featuresets, estimator=ELEProbDist):
    label_probdist = estimator(label_freqdist)
    feature_probdist = {}
    return NaiveBayesClassifier(label_probdist, feature_probdist)

In [ ]:
nltk.classify.accuracy(classifier, training_set)

In [47]:
# test
sent = 'This is a test sentence'
classifier.classify(extract_features(sent.split()))

'no_joy_word'

In [48]:
# test
sent = 'I have lost my sunglasses recently'
classifier.classify(extract_features(sent.split()))

'no_joy_word'

In [50]:
# test
sent = 'Jenny was so excited when we got her a book as a present'
classifier.classify(extract_features(sent.split()))

'joy_word'

In [53]:
# test
sent = 'Amazon rain forest is the most beautiful place in the world'
classifier.classify(extract_features(sent.split()))

'joy_word'